In [6]:
import geopandas
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
import seaborn as sb
import plotly
import dash
import plotly.plotly as ply
from plotly.graph_objs import *
from scipy.io import netcdf
%matplotlib inline

In [34]:
plotly.tools.set_credentials_file(username='jpterry', api_key='jC0hHrhZQhOys3ypgeg8')

In [122]:
origins = ["DR_Congo", "Afghanistan", "Syria", "Myanmar", "Sudan"]
destinations = ["USA", "UK", "France", "Canada", "Italy", "Germany"]

org_codes = ["COD", "AFG", "SYR", "MMR", "SDN"]
dest_codes = ["USA", "GBR", "FRA", "CAN", "ITA","DEU"]

years = [2000 + i for i in range(19)]

features = ['destination',
 'Origin',
 'applied',
 'accepted',
 'Rejected',
 'decisions',
 'Year',
 'Month',
 'Value',
 'deaths',
 'last_month',
 'two_months_ago',
 'next_month',
 'two_months_later']

pairs = []
for origin in origins:
    for destination in destinations:
        pairs.append((origin, destination))

In [3]:
# Read in data
groups = {}
for pair in pairs:
    groups[pair] = pd.read_csv('./data/grouped_' + str(pair) + '.csv', skipinitialspace=True)
    groups[pair] = groups[pair][features]

In [108]:
sent = np.array([])
for origin in origins:
    total = 0
    for destination in destinations:
        total += groups[(origin, destination)].Value.sum()
    sent = np.append(sent, total)
    
avg_sent = np.array([])
for origin in origins:
    total = 0
    for destination in destinations:
        total += groups[(origin, destination)].Value.sum()/(groups[pair].Year.max() - groups[pair].Year.min())
    avg_sent = np.append(avg_sent, total)

In [109]:
dead = np.array([])
for origin in origins:
    total = 0
    for destination in destinations:
        total += groups[(origin, destination)].deaths.sum()
    dead = np.append(dead, total)
    
avg_dead = np.array([])
for origin in origins:
    total = 0
    for destination in destinations:
        total += groups[(origin, destination)].deaths.sum()/(groups[pair].Year.max() - groups[pair].Year.min())
    avg_dead = np.append(avg_dead, total)

In [111]:
arrived = np.array([])
for destination in destinations:
    total = 0
    for origin in origins:
        total += groups[(origin, destination)].Value.sum()
    arrived = np.append(arrived, total)

avg_arrived = np.array([])
for origin in origins:
    total = 0
    for destination in destinations:
        total += groups[(origin, destination)].Value.sum()/(groups[pair].Year.max() - groups[pair].Year.min())
    avg_arrived = np.append(avg_arrived, total)

In [12]:
sent, arrived, dead

(array([ 78096., 337899., 568092.,  11655.,  59188.]),
 array([ 23848.,  95376., 119146.,  21075.,  28253., 767232.]),
 array([  950240., 10851299., 26689288.,   276607.,  2443237.]))

In [15]:
coordinates = {}
coordinates["Syria"] = (34.8, 38.997)
coordinates["DR_Congo"] = (-4.0383, 21.759)
coordinates["Afghanistan"] = (33.939, 67.71)
coordinates["Sudan"] = (12.863, 30.218)
coordinates["Myanmar"] = (21.9162, 95.956)

coordinates["USA"] = (37.09, -95.713)
coordinates["UK"] = (55.378, -3.436)
coordinates["France"] = (46.228, 2.214)
coordinates["Germany"] = (51.166, 10.4515)
coordinates["Italy"] = (41.872, 12.5674)
coordinates["Canada"] = (56.1304, -106.3468)

In [20]:
dest_lats = [coordinates[dest][0] for dest in destinations] 
dest_lons = [coordinates[dest][1] for dest in destinations] 
org_lats = [coordinates[org][0] for org in origins] 
org_lons = [coordinates[org][1] for org in origins]

In [38]:
org_texts =[origins[i] + 'Total deaths = ' + str(dead[i]) for i in range(len(dead))]

In [218]:
sent_each_year_by_orig = []
relative_sizes = []
for origin in origins:
    this_origin = []
    these_sizes = []
    for year in years:
        total = 0
        year = int(year)
        for destination in destinations:
            try:
                total += groups[(origin, destination)].query('Year == @year').Value.sum()
            except:
                total += 0
        this_origin.append(total)
    for i in range(len(years)):
        these_sizes.append(0.5e2*this_origin[i]/max(this_origin))
    sent_each_year_by_orig.append(this_origin)
    relative_sizes.append(these_sizes)

In [258]:
sent_each_year = [[] for i in range(len(years))]
year_sizes = [[] for i in range(len(years))]
for i in range(len(sent_each_year_by_orig)):
    for ii in range(len(sent_each_year_by_orig[i])):
        sent_each_year[ii].append(sent_each_year_by_orig[i][ii])
#         year_sizes[ii].append(relative_sizes[i][ii])
for i in range(len(sent_each_year)):
    for ii in range(len(sent_each_year[i])):
        if max(sent_each_year[i]) != 0:
            year_sizes[i].append(50*sent_each_year[i][ii]/max(sent_each_year[i]))
        else:
            year_sizes[i].append(0)

In [259]:
year_sizes

[[19.76754160233316, 50.0, 0.0, 4.704923657574198, 6.1760164693772515],
 [23.430634023854363, 50.0, 0.0, 4.526051475204017, 5.951035781544256],
 [43.243721259285465, 50.0, 0.0, 3.0022992571630702, 8.016448532012735],
 [0.0, 50.0, 0.0, 7.4312527280663465, 22.21737232649498],
 [0.0, 49.33026370866472, 23.69192130598577, 10.50648807032231, 50.0],
 [0.0, 50.0, 0.0, 8.146964856230031, 35.283546325878596],
 [50.0, 48.27633007600434, 0.0, 7.47828447339848, 20.779044516829533],
 [42.917004595836715, 50.0, 0.0, 9.367396593673966, 15.233846985671804],
 [26.556534508076357, 50.0, 0.0, 4.9192364170337735, 11.189427312775331],
 [20.9113750571037, 50.0, 0.0, 3.4547738693467336, 8.028780264961169],
 [21.236532071291915, 50.0, 0.0, 3.5948041486255162, 9.002114590675662],
 [20.13008672448299,
  50.0,
  17.545030020013343,
  2.605903935957305,
  11.749499666444295],
 [28.303137428192663,
  50.0,
  42.037118868758284,
  3.8223596995139197,
  8.948298718515245],
 [17.775665399239543,
  34.4945202415567,
 

In [210]:
sent_each_year

[[0.9218, 2.3316, 0.0, 0.2194, 0.288],
 [1.493, 3.186, 0.0, 0.2884, 0.3792],
 [1.956, 2.2616, 0.0, 0.1358, 0.3626],
 [0.0, 0.9164, 0.0, 0.1362, 0.4072],
 [0.0, 0.4714, 0.2264, 0.1004, 0.4778],
 [0.0, 0.5008, 0.0, 0.0816, 0.3534],
 [0.7368, 0.7114, 0.0, 0.1102, 0.3062],
 [0.635, 0.7398, 0.0, 0.1386, 0.2254],
 [0.7234, 1.362, 0.0, 0.134, 0.3048],
 [0.7324, 1.7512, 0.0, 0.121, 0.2812],
 [0.8436, 1.9862, 0.0, 0.1428, 0.3576],
 [0.9656, 2.3984, 0.8416, 0.125, 0.5636],
 [1.281, 2.263, 1.9026, 0.173, 0.405],
 [1.2716, 2.4676, 3.5768, 0.1034, 0.4846],
 [1.2542, 3.063, 9.5162, 0.0764, 0.9688],
 [1.0122, 8.2972, 33.7522, 0.0802, 1.9718],
 [0.8922, 27.87, 55.2702, 0.0864, 2.051],
 [0.9004, 5.0022, 8.5324, 0.0782, 1.6494],
 [0.0, 0.0, 0.0, 0.0, 0.0]]

### Start plots

#### Interactive Maps

In [98]:
data = [dict(
        type = 'choropleth',
        locations = org_codes,
        z = dead,
        text = origins,
        colorscale = [[0,"rgb(139,0,0)"],[0.35,"rgb(255,0,0)"],[0.5,"rgb(255,140,0)"],\
            [0.6,"rgb(255,165,0)"],[0.7,"rgb(255,255,0)"], [1, "rgb(255,255,204)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '',
            title = 'Conflict Deaths'),
      ) ]

layout = dict(
    title = 'Total Conflict Deaths 2000 - 2018',
    geo = dict(
        showframe = False, 
        showcoastlines = True,
        showcountries = True, 
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
ply.iplot(fig, validate=False, filename='deaths_world' )

In [121]:
data = [dict(
        type = 'choropleth',
        locations = org_codes,
        z = avg_dead,
        text = origins,
        colorscale = [[0,"rgb(139,0,0)"],[0.35,"rgb(255,0,0)"],[0.5,"rgb(255,140,0)"],\
            [0.6,"rgb(255,165,0)"],[0.7,"rgb(255,255,0)"], [1, "rgb(255,255,204)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '',
            title = 'Conflict Deaths'),
      ) ]

layout = dict(
    title = 'Conflict Deaths per Year',
    geo = dict(
        showframe = False, 
        showcoastlines = True,
        showcountries = True, 
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
ply.iplot(fig, validate=False, filename='avg_deaths_world' )

In [120]:
data = [dict(
        type = 'choropleth',
        locations = org_codes,
        z = sent,
        text = origins,
        colorscale = [[0,"rgb(255,0,0)"],[0.35,"rgb(255,99,71))"],[0.5,"rgb(255,140,0)"],\
            [0.6,"rgb(255,165,0)"],[0.7,"rgb(255,215,0"],[1,"rgb(255,255,102)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '',
            title = 'Total Refugees Deaths'),
      ) ]

layout = dict(
    title = 'Total Refugees 2000 - 2018',
    geo = dict(
        showframe = False, 
        showcoastlines = True,
        showcountries = True, 
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        projection = dict(
            type = 'Mercator'
        )
    )
)

data2 = [dict(
        type = 'choropleth',
        locations = dest_codes,
        z = arrived,
        text = destinations,
        colorscale = [[0,"rgb(255,0,0)"],[0.35,"rgb(255,99,71))"],[0.5,"rgb(255,140,0)"],\
            [0.6,"rgb(255,165,0)"],[0.7,"rgb(255,255,0)"], [1, "rgb(255,255,224)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '',
            title = 'Refugees'),
      ) ]

layout2 = dict(
    title = 'Total Refugees 2000 - 2018',
    geo = dict(
        showframe = False, 
        showcoastlines = True,
        showcountries = True, 
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        projection = dict(
            type = 'Mercator'
        )
    )
)



fig = dict(data=data, layout=layout)
fig2 = dict(data=data2, layout=layout2)
ply.iplot(fig2, validate=False, filename='total_refugees_accepted' )

In [118]:
ply.iplot(fig, validate=False, filename='total_refugees_sent' )

In [112]:
data = [dict(
        type = 'choropleth',
        locations = org_codes,
        z = avg_sent,
        text = origins,
        colorscale = [[0,"rgb(255,0,0)"],[0.35,"rgb(255,99,71))"],[0.5,"rgb(255,140,0)"],\
            [0.6,"rgb(255,165,0)"],[0.7,"rgb(255,215,0"],[1,"rgb(255,255,102)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '',
            title = 'Refugees Sent per Year'),
      ) ]

layout = dict(
    title = 'Refugees Sent per Year',
    geo = dict(
        showframe = False, 
        showcoastlines = True,
        showcountries = True, 
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        projection = dict(
            type = 'Mercator'
        )
    )
)

In [117]:
fig = dict(data=data, layout=layout)
ply.iplot(fig, validate=False, filename='avg_refugees_sent' )

In [114]:
data = [dict(
        type = 'choropleth',
        locations = dest_codes,
        z = avg_arrived,
        text = destinations,
        colorscale = [[0,"rgb(255,0,0)"],[0.35,"rgb(255,99,71))"],[0.5,"rgb(255,140,0)"],\
            [0.6,"rgb(255,165,0)"],[0.7,"rgb(255,215,0"],[1,"rgb(255,255,102)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '',
            title = 'Refugees Accepted per Year'),
      ) ]

layout = dict(
    title = 'Refugees Refugees per Year',
    geo = dict(
        showframe = False, 
        showcoastlines = True,
        showcountries = True, 
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        projection = dict(
            type = 'Mercator'
        )
    )
)

In [116]:
fig = dict(data=data, layout=layout)
ply.iplot(fig, validate=False, filename='avg_refugees_accepted' )

#### Refugees each year

In [267]:
import plotly.graph_objs as go
sent_year = []
# colors = ['rgb(239,243,255)','rgb(189,215,231)','rgb(107,174,214)','rgb(33,113,181)']
# colors = 'inferno'

labels = []
for i in range(len(sent_each_year)):
    labels.append([])
    for ii in range(len(origins)):
        labels[i].append(str(sent_each_year[i][ii]) + ' ' + origins[ii])

cmap = plt.get_cmap('BuPu')
colors = cmap(np.linspace(0, 1.0, 18))

for i in range(len(years)-1):
    sent_year.append(go.Scattergeo(
        lon = org_lons,
        lat = org_lats,
        text = labels[i],
        name = str(years[i]),
        marker = dict(
            size = year_sizes[i],
            color = colors[i]
#             line = dict(width = 0)
        ),
    ) )

# cases[0]['text'] = df[ df['Month'] == 9 ]['Value'].map('{:.0f}'.format).astype(str)+' '+\
#     df[ df['Month'] == 9 ]['Country']
# cases[0]['mode'] = 'markers+text'
# cases[0]['textposition'] = 'bottom center'

inset = [
    go.Choropleth(
#         locationmode = 'country codes',
        locations = org_codes,
        z = sent_each_year,
#         text = df[ df['Month'] == 9 ]['Country'],
        colorscale = [[0,'rgb(0, 0, 0)'],[1,'rgb(0, 0, 0)']],
        autocolorscale = False,
        showscale = False,
        geo = 'geo'
    ),
    go.Scattergeo(
        lon = [21.0936],
        lat = [7.1881],
        text = ['World'],
        mode = 'text',
        showlegend = False,
        geo = 'geo'
    )
]

layout = go.Layout(
    title = 'Regufees each year',
    geo = dict(
        resolution = 50,
        scope = 'asia',
        showframe = False,
        showcoastlines = True,
        showland = True,
        showcountries=True,
        landcolor = "rgb(229, 229, 229)",
        countrycolor = "rgb(255, 255, 255)" ,
        coastlinecolor = "rgb(255, 255, 255)",
        projection = dict(
            type = 'mercator'
        )#,
#         lonaxis = dict( range= [ -15.0, -5.0 ] ),
#         lataxis = dict( range= [ 0.0, 12.0 ] ),
#         domain = dict(
#             x = [ 0, 1 ],
#             y = [ 0, 1 ]
#         )
    )#,
#     geo2 = dict(
#         scope = 'world',
#         showframe = False,
#         showland = True,
#         landcolor = "rgb(229, 229, 229)",
#         showcountries = False,
#         domain = dict(
#             x = [ 0, 0.6 ],
#             y = [ 0, 0.6 ]
#         ),
#         bgcolor = 'rgba(255, 255, 255, 0.0)',
#     ),
#     legend = dict(
#            traceorder = 'reversed'
#     )
)

fig = go.Figure(layout=layout, data=sent_year)
ply.iplot(fig, validate=False, filename='Refugees each year')

PlotlyRequestError: Hi there, you've reached the threshold of 100 combined image exports and chart saves per 24h period. If you need to raise your daily limit, please consider upgrading to a paid plan
(see: https://plot.ly/products/cloud ).

In [240]:
labels

[['4609.0 DR_Congo',
  '11658.0 Afghanistan',
  '0.0 Syria',
  '1097.0 Myanmar',
  '1440.0 Sudan'],
 ['7465.0 DR_Congo',
  '15930.0 Afghanistan',
  '0.0 Syria',
  '1442.0 Myanmar',
  '1896.0 Sudan'],
 ['9780.0 DR_Congo',
  '11308.0 Afghanistan',
  '0.0 Syria',
  '679.0 Myanmar',
  '1813.0 Sudan'],
 ['0.0 DR_Congo',
  '4582.0 Afghanistan',
  '0.0 Syria',
  '681.0 Myanmar',
  '2036.0 Sudan'],
 ['0.0 DR_Congo',
  '2357.0 Afghanistan',
  '1132.0 Syria',
  '502.0 Myanmar',
  '2389.0 Sudan'],
 ['0.0 DR_Congo',
  '2504.0 Afghanistan',
  '0.0 Syria',
  '408.0 Myanmar',
  '1767.0 Sudan'],
 ['3684.0 DR_Congo',
  '3557.0 Afghanistan',
  '0.0 Syria',
  '551.0 Myanmar',
  '1531.0 Sudan'],
 ['3175.0 DR_Congo',
  '3699.0 Afghanistan',
  '0.0 Syria',
  '693.0 Myanmar',
  '1127.0 Sudan'],
 ['3617.0 DR_Congo',
  '6810.0 Afghanistan',
  '0.0 Syria',
  '670.0 Myanmar',
  '1524.0 Sudan'],
 ['3662.0 DR_Congo',
  '8756.0 Afghanistan',
  '0.0 Syria',
  '605.0 Myanmar',
  '1406.0 Sudan'],
 ['4218.0 DR_Congo',


In [238]:
a = []
a == True

False